In [ ]:
#show table in database
postgres=# \dt

#describe table
postgres=# \d tablename

#show top rows in a table
postgres=# TABLE tablename



# 1  SQL DML: Data Manipulation Language

# 2 Inserting Data: Form One

```sql
INSERT INTO table (column list) VALUES (first row of values), ...
```

```sql
INSERT INTO "movies"("name", "release_date") VALUES
    ('EPISODE 4 - a new hope', '1977-05-25'),
    ('EPISODE 5 - the empire strikes back', '1980-05-17'),
    ('EPISODE 6 - return of the jedi', '1983-05-25');

INSERT INTO "movies"("id", "name", "release_date") VALUES
    (1, 'EPISODE 4 - a new hope', '1977-05-25'),
    (2, 'EPISODE 5 - the empire strikes back', '1980-05-17'),
    (3, 'EPISODE 6 - return of the jedi', '1983-05-25');

INSERT INTO "movies"("name", "release_date") VALUES
    (DEFAULT, 'EPISODE 4 - a new hope', '1977-05-25'),
    (DEFAULT, 'EPISODE 5 - the empire strikes back', '1980-05-17'),
    (DEFAULT, 'EPISODE 6 - return of the jedi', '1983-05-25');

    ```

# 3 Inserting Data: Form Two

```sql
INSERT INTO table_name ("column list in the order it's returned by the SELECT") SELECT … ;

INSERT INTO "categories" ("name") SELECT DISTINCT "category" FROM "books" ;

CREATE TABLE "categories" (
    "id" SERIAL,
    "name" VARCHAR
);

INSERT INTO "categories" ("name") 
    SELECT DISTINCT "category" FROM "posts" ;
```






# 6 Inserting Data: Exercise Solution

```sql
-- Migrate people
INSERT INTO "people" ("first_name", "last_name")
  SELECT "first_name", "last_name" FROM "denormalized_people";

SELECT * FROM "people";

  -- Migrate people's emails using the correct ID
INSERT INTO "people_emails"
SELECT "p"."id", REGEXP_SPLIT_TO_TABLE("dn"."emails", ',')
FROM "denormalized_people" "dn"
JOIN "people" "p" ON (
  "dn"."first_name" = "p"."first_name"
  AND "dn"."last_name" = "p"."last_name"
);
```


# 7 Updating Data in Postgres

```sql
--Create users table

CREATE TABLE users (
    id SERIAL,
    name VARCHAR,
    happiness_level SMALLINT,
    mood VARCHAR
);

-- Insert data into users table

INSERT INTO users (name, happiness_level)
VALUES 
    ('User4', 12),
    ('User1', 50),
    ('User2', 75),
    ('User3', 89);

-- Create posts table

CREATE TABLE posts (
    id SERIAL,
    title TEXT,
    category VARCHAR
);

-- Insert data into posts table

INSERT INTO posts (title, category)
VALUES 
    ('First posts', 'General'),
    ('Second posts', 'Travel'),
    ('Third posts', 'Equipment'),
    ('4th posts', 'Travel'),
    ('lorem ipsum', 'Tips'),
    ('Amother post', 'Travel');

-- Create categories table

CREATE TABLE categories (
    id SERIAL,
    name VARCHAR
);

-- Insert data into categories table

INSERT INTO categories (name)
VALUES 
    ('Travel'),
    ('Equipment'),
    ('General'),
    ('Tips');


-- Update mood based on happiness_level

UPDATE "users" SET "mood" = 'LOW' WHERE "happiness_level" < 33;
UPDATE "users" SET "mood" = 'Average' WHERE "happiness_level" BETWEEN 33 AND 65;
UPDATE "users" SET "mood" = 'Good' WHERE "happiness_level" >= 66;

-- Select all records from the users table

SELECT * FROM users;

-- Update mood to 'Excellent' for all users

UPDATE "users" SET "mood" = 'Excellent';

-- Select all records from the users table

SELECT * FROM users;

-- Update mood to 'Excellent' and set happiness_level to 100 for all users

UPDATE "users" SET "mood" = 'Excellent', "happiness_level" = 100;

-- Select all records from the users table

SELECT * FROM us



-- Add category_id column to posts table

ALTER TABLE "posts" ADD COLUMN "category_id" INTEGER;

-- Display the structure of the posts table

TABLE posts;

-- Update category_id in posts table based on category names

UPDATE "posts" SET "category_id" = (
    SELECT "id" FROM "categories" WHERE "categories"."name" = "posts"."category"
);

-- Display the structure of the posts table

TABLE posts;



```

# 10 Updating Data: Solution

```sql
-- Update the last_name column to be capitalized
UPDATE "people" SET "last_name" =
  SUBSTR("last_name", 1, 1) ||
  LOWER(SUBSTR("last_name", 2));



-- Change the born_ago column to date_of_birth
ALTER TABLE "people" ADD column "date_of_birth" DATE;

UPDATE "people" SET "date_of_birth" = 
  (CURRENT_TIMESTAMP - "born_ago"::INTERVAL)::DATE;

ALTER TABLE "people" DROP COLUMN "born_ago";
```

# 11 Deleting Data in Postgres


The basic syntax for deleting rows from a table is **DELETE FROM table_name WHERE** …. Just like **SELECT** and **UPDATE**, omitting the **WHERE** clause will delete all rows from the table. Again, this is rarely what you want to do! Contrary to **TRUNCATE TABLE**, doing a **DELETE** without a **WHERE** won't allow you to restart the sequence if you have one in your table. More importantly, in a future lesson we'll learn about indexing as a way to make queries perform faster in the presence of large amounts of data. Running TRUNCATE will also clear these indexes, which will further accelerate queries once new data gets inserted in that table.

# 12 Data Manipulation: Transactions

```sql
SELECT
```

# 14 Data Manipulation Exercise: Solution

```sql
-- Do everything in a transaction
BEGIN;


-- Remove all users from New York and California
DELETE FROM "user_data" WHERE "state" IN ('NY', 'CA');


-- Split the name column in first_name and last_name
ALTER TABLE "user_data"
  ADD COLUMN "first_name" VARCHAR,
  ADD COLUMN "last_name" VARCHAR;

UPDATE "user_data" SET
  "first_name" = SPLIT_PART("name", ' ', 1),
  "last_name" = SPLIT_PART("name", ' ', 2);

ALTER TABLE "user_data" DROP COLUMN "name";


-- Change from state to state_id
CREATE TABLE "states" (
  "id" SMALLSERIAL,
  "state" CHAR(2)
);

INSERT INTO "states" ("state")
  SELECT DISTINCT "state" FROM "user_data";

ALTER TABLE "user_data" ADD COLUMN "state_id" SMALLINT;

UPDATE "user_data" SET "state_id" = (
  SELECT "s"."id"
  FROM "states" "s"
  WHERE "s"."state" = "user_data"."state"
);

ALTER TABLE "user_data" DROP COLUMN "state";
```

# 16 Glossary

| Key Term            | Definition |
|---------------------|------------|
| **INSERT ... VALUES** | One form of adding data into a table; used when introducing new data in a table. This data would come from an external source like an application. |
| **INSERT ... SELECT** | One form of adding data into a table; used when taking already existing data from a table and migrating it — most often with some modifications or clean-ups — into an already existing table. |
| **UPDATE**           | Used to update rows of data within a given column with new values. |
| **DELETE**           | Used to delete some portion of data from a table. |
| **BEGIN**            | Starts a transaction. |
| **COMMIT**           | Tells the system to attempt to complete the transaction (make the requested changes). Similar functionality is also achieved with END. |
| **ROLLBACK**         | Tells the system to not commit any changes as part of the current transaction, discarding the changes. |
| **ACID**             | An acronym that describes the transactional guarantees provided by a relational database. |
| **Atomicity**        | The database guarantees that a transaction will either register all the commands in a transaction, or none of them. |
| **Consistency**      | The database guarantees that a successful transaction will leave the data in a consistent state, one that obeys all the rules that you've set up. |
| **Isolation**        | The database guarantees that concurrent transactions don't "see each other" until they are committed. Committing a transaction is a command that tells the database to execute all the commands we passed to it since we started that transaction. |
| **Durability**       | The database guarantees that once it accepts a transaction and returns a success, the changes introduced by the transaction will be permanently stored on disk, even if the database crashes right after the success response. |


```sql

```